<a href="https://colab.research.google.com/github/nicolacalzone/CV_Lab8/blob/main/NLP_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Introduction

Transition-based dependency parsing is one of the most popular methods for implementing a dependency parsers.  We use here the **Arc-Eager** model. We augment the parser with neural machinery for contextual word embeddings and for choosing the most appropriate parser actions.  

We implement the following features:
* LSTM representation for stack tokens
* MLP for next transition classification, based on two top-most stack tokens and first token in the buffer
* training under static oracle

In [ ]:
!pip install datasets  # huggingface library with dataset
!pip install conllu    # aux library for processing CoNLL-U format

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 23.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 26.6 MB/s eta 0:00:00


In [ ]:
import torch
import torch.nn as nn
from functools import partial
from datasets import load_dataset

## Arc-Eager

The **configuration** of the arc-eager parser is a triple of the form $( \sigma, \beta, A)$ where:

* $\sigma$ is the stack;
* $\beta$ is the input buffer;
* $A$ is a set of arcs constructed so far.

The parser can perform four types of **actions** (transitions):

* **shift**, which removes the first node in the buffer and pushes it onto the stack;
* **left-arc**, which adds the arc $(\beta_1 \rightarrow \sigma_1)$ to $A$, where $\sigma_1$ is the node on top of the stack and $\beta_1$ is the first node in the buffer, and pops the stack. It has as a precondition that the token $\sigma_1$ is not the artificial root node 0 and does not already have a head;
* **right-arc**, which adds the arc $(\sigma_1 \rightarrow \beta_1)$ to $A$, where $\sigma_1$ is the node on top of the stack and $\beta_1$ is the first node in the buffer, and pushes the node $\beta_1$ onto the stack;
* **reduce**, which pops the stack and is subject to the preconditions that the top token has a head.

Let $w = w_0 w_1 \cdots w_{n}$ be the input sentence, with $w_0$ the special symbol `<ROOT>`.
Stack and buffer are implemented as lists of integers, where `j` represents word $w_j$. Top-most stack token is at the right-end of the list; first buffer token is at the left-end of the list.
Set $A$ is implemented as an array `arcs` of size $n+1$ such that if arc $(w_i \rightarrow w_j)$ is in $A$ then `arcs[j]=i`, and if $w_j$ is still missing its head node in the tree under construction, then `arcs[j]=-1`. We always have `arcs[0]=-1`. We use this representation also for complete dependency trees.



In [ ]:
class ArcEager:
  def __init__(self, sentence):
    self.sentence = sentence
    self.buffer = [i for i in range(len(self.sentence))]
    self.stack = []
    self.arcs = [-1 for _ in range(len(self.sentence))]

    # shift move to initialize the stack
    self.shift()

  def shift(self):
    b1 = self.buffer[0]
    self.buffer = self.buffer[1:]
    self.stack.append(b1)

  def left_arc(self):
    o1 = self.stack.pop()
    o2 = self.buffer[0]
    self.arcs[o1] = o2
    #if len(self.stack) < 2 and len(self.buffer) > 0:
    #  self.shift()

  def right_arc(self):
    o1 = self.buffer[0]
    self.buffer = self.buffer[1:]
    o2 = self.stack.pop()
    self.arcs[o1] = o2
    self.stack.append(o2)
    self.stack.append(o1)
    #if len(self.stack) < 2 and len(self.buffer) > 0:
    #  self.shift()

  def reduce(self):
    o1 = self.stack.pop()

  def is_tree_final(self):
    return len(self.buffer) == 0

  def print_configuration(self):
    s = [self.sentence[i] for i in self.stack]
    b = [self.sentence[i] for i in self.buffer]
    print(s, b)
    print(self.arcs)

In [ ]:
sentence = ["<ROOT>", "He", "began", "to", "write", "again", "."]
gold = [-1, 2, 0, 4, 2, 4, 2 ]

parser = ArcEager(sentence)
parser.print_configuration()

['<ROOT>'] ['He', 'began', 'to', 'write', 'again', '.']
[-1, -1, -1, -1, -1, -1, -1]


In [ ]:
parser.left_arc()
parser.print_configuration()

[] ['He', 'began', 'to', 'write', 'again', '.']
[1, -1, -1, -1, -1, -1, -1]


In [ ]:
parser.shift()
parser.print_configuration()

['He'] ['began', 'to', 'write', 'again', '.']
[1, -1, -1, -1, -1, -1, -1]


In [ ]:
parser.right_arc()
parser.print_configuration()

['He', 'began'] ['to', 'write', 'again', '.']
[1, -1, 1, -1, -1, -1, -1]


In [ ]:
parser.reduce()
parser.print_configuration()

['He'] ['to', 'write', 'again', '.']
[1, -1, 1, -1, -1, -1, -1]


## Oracle

A **static oracle** maps parser configurations $c$ into actions, and it does so by looking into the gold (reference) tree for the sentence at hand. If $c$ does not contain any mistake, then the action provided by the oracle for $c$ is guaranteed to be correct. Furthermore, in cases where there is more than one correct action for $c$, the oracle always chooses a single action, called the **canonical** action.

We use here the static oracle for the arc-eager parser which is based on the following conditions:
* set $A$ in configuration $c$ does not contain any wrong dependency
* left-arc has precedence over other actions, and can be done only if it constructs a gold dependency from $\beta_1$ to $\sigma_1$
* right-arc can be done only if it constructs a gold dependency from $\sigma_1$ to $\beta_1$
* reduce transition has precedence over shift, and can be done if there exists a node $k$ such that $k < \sigma_1$ such that either $(k, \beta_1)$ or $(\beta_1, k)$ is in the gold tree
* shift transition has lowest precedence, and can be done if the buffer is not empty


It is not difficult to see that the four actions above are mutually exclusive.

Here is the implementation of the Arc-Eager Oracle:


In [ ]:
class Oracle:
  def __init__(self, parser, gold_tree):
    self.parser = parser
    self.gold = gold_tree

  def is_left_arc_gold(self):
    o1 = self.parser.stack[len(self.parser.stack)-1]
    o2 = self.parser.buffer[0]

    if self.gold[o1] == o2:
      return True
    return False

  def is_right_arc_gold(self):
    o1 = self.parser.stack[len(self.parser.stack)-1]
    o2 = self.parser.buffer[0]

    if self.gold[o2] == o1:
      return True
    return False


  def is_reduce_gold(self):
    o1 = self.parser.stack[len(self.parser.stack)-1]
    o2 = self.parser.buffer[0]

    for k in range(0,o1):
      if self.gold[k] == o2 or self.gold[o2] == k:
        return True
    return False


  def is_shift_gold(self):
    if len(self.parser.buffer) == 0:
      return False
    if  self.is_left_arc_gold() == False and self.is_right_arc_gold() == False and self.is_reduce_gold() == False:
      return True
    return False


In [ ]:
sentence = ["<ROOT>", "He", "began", "to", "write", "again", "."]
gold = [-1, 2, 0, 4, 2, 4, 2 ]

parser = ArcEager(sentence)
oracle = Oracle(parser, gold)

parser.print_configuration()

['<ROOT>'] ['He', 'began', 'to', 'write', 'again', '.']
[-1, -1, -1, -1, -1, -1, -1]


In [ ]:
while not parser.is_tree_final():
  parser.print_configuration()
  if oracle.is_left_arc_gold():
    parser.left_arc()
    print("left arc")
  elif oracle.is_right_arc_gold():
    parser.right_arc()
    print("right arc")
  elif oracle.is_reduce_gold():
    parser.reduce()
    print("reduce")
  elif oracle.is_shift_gold():
    print("shift")
    parser.shift()


print(parser.arcs)
print(gold)

['<ROOT>'] ['He', 'began', 'to', 'write', 'again', '.']
[-1, -1, -1, -1, -1, -1, -1]
shift
['<ROOT>', 'He'] ['began', 'to', 'write', 'again', '.']
[-1, -1, -1, -1, -1, -1, -1]
left arc
['<ROOT>'] ['began', 'to', 'write', 'again', '.']
[-1, 2, -1, -1, -1, -1, -1]
right arc
['<ROOT>', 'began'] ['to', 'write', 'again', '.']
[-1, 2, 0, -1, -1, -1, -1]
shift
['<ROOT>', 'began', 'to'] ['write', 'again', '.']
[-1, 2, 0, -1, -1, -1, -1]
left arc
['<ROOT>', 'began'] ['write', 'again', '.']
[-1, 2, 0, 4, -1, -1, -1]
right arc
['<ROOT>', 'began', 'write'] ['again', '.']
[-1, 2, 0, 4, 2, -1, -1]
right arc
['<ROOT>', 'began', 'write', 'again'] ['.']
[-1, 2, 0, 4, 2, 4, -1]
reduce
['<ROOT>', 'began', 'write'] ['.']
[-1, 2, 0, 4, 2, 4, -1]
reduce
['<ROOT>', 'began'] ['.']
[-1, 2, 0, 4, 2, 4, -1]
right arc
[-1, 2, 0, 4, 2, 4, 2]
[-1, 2, 0, 4, 2, 4, 2]


## Dataset

We use the huggingface [datasets](https://github.com/huggingface/datasets) library, and train on the English [treebank](https://huggingface.co/datasets/viewer/?dataset=universal_dependencies) from the Universal Dependency project.  

Variable `dataset` below is an array recording the training set.
Each element `dataset[i]` is a dictionary, recording key/annotation pairs for the $(i+1)$-th sentence.  Possible annotations are tokens, lemmas, PoS tags, dependency tree, etc., which can be accessed using the appropriate key.


In [ ]:
train_dataset = load_dataset('universal_dependencies', 'en_lines', split="train")
dev_dataset = load_dataset('universal_dependencies', 'en_lines', split="validation")
test_dataset = load_dataset('universal_dependencies', 'en_lines', split="test")

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/3176 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1032 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1035 [00:00<?, ? examples/s]

Dataset universal_dependencies downloaded and prepared to /root/.cache/huggingface/datasets/universal_dependencies/en_lines/2.7.0/1ac001f0e8a0021f19388e810c94599f3ac13cc45d6b5b8c69f7847b2188bdf7. Subsequent calls will reuse this data.


## Create training data and iterable dataloaders

To run the arc-eager parser we need a **classifier** that looks at some of the content of the current parser configuration and selects an approapriate action.  In order to train the classifier, we need to convert the gold trees in our treebank into several pairs of the form configuration/gold action.

First of all, we need to preprocess the training set.
Then we remove non-projective trees and create a dictionary of word/index pairs, to be used later when creating word embeddings.  Words that have less than three occurrences are not encoded and will later be mapped to special token `<unk>`.

In [ ]:
# the function returns whether a tree is projective or not. It is currently
# implemented inefficiently by brute checking every pair of arcs.
def is_projective(tree):
  for i in range(len(tree)):
    if tree[i] == -1:
      continue
    left = min(i, tree[i])
    right = max(i, tree[i])

    for j in range(0, left):
      if tree[j] > left and tree[j] < right:
        return False
    for j in range(left+1, right):
      if tree[j] < left or tree[j] > right:
        return False
    for j in range(right+1, len(tree)):
      if tree[j] > left and tree[j] < right:
        return False

  return True

# the function creates a dictionary of word/index pairs: our embeddings vocabulary
# threshold is the minimum number of appearance for a token to be included in the embedding list
def create_dict(dataset, threshold=3):
  dic = {}  # dictionary of word counts
  for sample in dataset:
    for word in sample['tokens']:
      if word in dic:
        dic[word] += 1
      else:
        dic[word] = 1

  map = {}  # dictionary of word/index pairs. This is our embedding list
  map["<pad>"] = 0
  map["<ROOT>"] = 1
  map["<unk>"] = 2 #used for words that do not appear in our list

  next_indx = 3
  for word in dic.keys():
    if dic[word] >= threshold:
      map[word] = next_indx
      next_indx += 1

  return map

In [ ]:
train_dataset = [sample for sample in train_dataset if is_projective([-1] + [int(head) for head in sample["head"]])]
emb_dictionary = create_dict(train_dataset)

The next function is used to process our data and create the actual training samples.

For each sentence in the dataset, we use our oracle to compute the canonical action sequence leading to the gold tree.  We then pair configurations and canonical actions.  Since our neural classifier will look only into $\sigma_1$, $\sigma_2$ and $\beta_1$, we do not have to record the full parser configuration.   

In [ ]:
def process_sample(sample, get_gold_path = False):

  # put sentence and gold tree in our format
  sentence = ["<ROOT>"] + sample["tokens"]
  gold = [-1] + [int(i) for i in sample["head"]]  #heads in the gold tree are strings, we convert them to int

  # embedding ids of sentence words
  enc_sentence = [emb_dictionary[word] if word in emb_dictionary else emb_dictionary["<unk>"] for word in sentence]

  # gold_path and gold_moves are parallel arrays whose elements refer to parsing steps
  gold_path = []   # record two topmost stack tokens and first buffer token for current step
  gold_moves = []  # contains oracle (canonical) move for current step: 0 is left, 1 right, 2 reduce, 3 shift

  if get_gold_path:  # only for training
    parser = ArcEager(sentence)
    oracle = Oracle(parser, gold)

    while not parser.is_tree_final():

      # save configuration
      configuration = [parser.stack[len(parser.stack)-2], parser.stack[len(parser.stack)-1]]
      if len(parser.buffer) == 0:
        configuration.append(-1)
      else:
        configuration.append(parser.buffer[0])
      gold_path.append(configuration)

      # save gold move
      if oracle.is_left_arc_gold():
        gold_moves.append(0)
        parser.left_arc()
      elif oracle.is_right_arc_gold():
        parser.right_arc()
        gold_moves.append(1)
      elif oracle.is_reduce_gold():
        parser.reduce()
        gold_moves.append(2)
      elif oracle.is_shift_gold():
        parser.shift()
        gold_moves.append(3)

  return enc_sentence, gold_path, gold_moves, gold

We now need to batch the training data.
This function is only used by the BiLSTM model. Later on, a very similar function will be used to batch the training data for the BERT model.

In [ ]:
def prepare_batch(batch_data, get_gold_path=False):
  data = [process_sample(s, get_gold_path=get_gold_path) for s in batch_data]
  # sentences, paths, moves, trees are parallel arrays, each element refers to a sentence
  sentences = [s[0] for s in data]
  paths = [s[1] for s in data]
  moves = [s[2] for s in data]
  trees = [s[3] for s in data]

  return sentences, paths, moves, trees

We create the data loaders for training, test and validation (developement) sets.

In [ ]:
BATCH_SIZE = 32

train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=partial(prepare_batch, get_gold_path=True))
dev_dataloader = torch.utils.data.DataLoader(dev_dataset, batch_size=BATCH_SIZE, shuffle=False, collate_fn=partial(prepare_batch))
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, collate_fn=partial(prepare_batch))

In [ ]:
for batch in train_dataloader:
  for l in batch[0]:
    print(l)
  break


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## Create neural network model  

First of all, we define some parameters.

In [ ]:
EMBEDDING_SIZE = 200
LSTM_SIZE = 200
LSTM_LAYERS = 1
MLP_SIZE = 200
DROPOUT = 0.2
EPOCHS = 15
LR = 0.001   # learning rate

We define the class Net to create a neural network model for arc-eager dependency parsing.

In [ ]:
class Net(nn.Module):

  def __init__(self, device):
    super(Net, self).__init__()
    self.device = device
    self.embeddings = nn.Embedding(len(emb_dictionary), EMBEDDING_SIZE, padding_idx=emb_dictionary["<pad>"])

    # initialize bi-LSTM
    self.lstm = nn.LSTM(EMBEDDING_SIZE, LSTM_SIZE, num_layers = LSTM_LAYERS, bidirectional=True, dropout=DROPOUT)

    # initialize feedforward
    self.w1 = torch.nn.Linear(6*LSTM_SIZE, MLP_SIZE, bias=True)
    self.activation = torch.nn.Tanh()
    self.w2 = torch.nn.Linear(MLP_SIZE, 4, bias=True)
    self.softmax = torch.nn.Softmax(dim=-1)

    self.dropout = torch.nn.Dropout(DROPOUT)


  def forward(self, x, paths):
    # get the embeddings
    x = [self.dropout(self.embeddings(torch.tensor(i).to(self.device))) for i in x]

    # run the bi-lstm
    h = self.lstm_pass(x)

    # for each parser configuration that we need to score we arrange from the
    # output of the bi-lstm the correct input for the feedforward
    mlp_input = self.get_mlp_input(paths, h)

    # run the feedforward and get the scores for each possible action
    out = self.mlp(mlp_input)

    return out

  def lstm_pass(self, x):
    x = torch.nn.utils.rnn.pack_sequence(x, enforce_sorted=False)
    h, (h_0, c_0) = self.lstm(x)
    h, h_sizes = torch.nn.utils.rnn.pad_packed_sequence(h) # size h: (length_sentences, batch, output_hidden_units)
    return h

  def get_mlp_input(self, configurations, h):
    mlp_input = []
    zero_tensor = torch.zeros(2*LSTM_SIZE, requires_grad=False).to(self.device)
    for i in range(len(configurations)): # for every sentence in the batch
      for j in configurations[i]: # for each configuration of a sentence
        mlp_input.append(torch.cat([zero_tensor if j[0]==-1 else h[j[0]][i], zero_tensor if j[1]==-1 else h[j[1]][i], zero_tensor if j[2]==-1 else h[j[2]][i]]))
    mlp_input = torch.stack(mlp_input).to(self.device)
    return mlp_input

  def mlp(self, x):
    return self.softmax(self.w2(self.dropout(self.activation(self.w1(self.dropout(x))))))

  # we use this function at inference time. We run the parser and at each step
  # we pick as next move the one with the highest score assigned by the model
  def infere(self, x):

    parsers = [ArcEager(i) for i in x]

    x = [self.embeddings(torch.tensor(i).to(self.device)) for i in x]

    h = self.lstm_pass(x)

    while not self.parsed_all(parsers):
      # get the current configuration and score next moves
      configurations = self.get_configurations(parsers)
      mlp_input = self.get_mlp_input(configurations, h)
      mlp_out = self.mlp(mlp_input)
      # take the next parsing step
      self.parse_step(parsers, mlp_out)

    # return the predicted dependency tree
    return [parser.arcs for parser in parsers]

  def get_configurations(self, parsers):
    configurations = []

    for parser in parsers:
      if parser.is_tree_final():
        conf = [-1, -1, -1]
      else:
        conf = [parser.stack[len(parser.stack)-2], parser.stack[len(parser.stack)-1]]
        if len(parser.buffer) == 0:
          conf.append(-1)
        else:
          conf.append(parser.buffer[0])
      configurations.append([conf])

    return configurations

  def parsed_all(self, parsers):
    for parser in parsers:
      if not parser.is_tree_final():
        return False
    return True

  # In this function we select and perform the next move according to the scores obtained.
  # We need to be careful and select correct moves, e.g. don't do a shift if the buffer
  # is empty or a left arc if σ2 is the ROOT. For clarity sake we didn't implement
  # these checks in the parser so we must do them here. This renders the function quite ugly
  def parse_step(self, parsers, moves):
    moves_argm = moves.argmax(-1)

    for i in range(len(parsers)):
      # If the following condition is skipped the buffer is always non-empty!
      if parsers[i].is_tree_final():
        continue
      else:
        # Left Arc
        if moves_argm[i] == 0:
          if parsers[i].stack[len(parsers[i].stack)-1] != 0 and parsers[i].arcs[len(parsers[i].stack) - 1] == -1:
            parsers[i].left_arc()
          else:
            if len(parsers[i].stack) > 0:
              parsers[i].right_arc()
            else:
              parsers[i].shift()

        # Right Arc
        elif moves_argm[i] == 1:
          if len(parsers[i].stack) > 0:
            parsers[i].right_arc()
          else:
            parsers[i].shift()

        # Reduce
        elif moves_argm[i] == 2:
          if len(parsers[i].stack) > 1 and parsers[i].arcs[len(parsers[i].stack) - 1] != -1:
            parsers[i].reduce()
          else:
            if len(parsers[i].stack) > 1:
              parsers[i].left_arc()
            elif len(parsers[i].stack) > 0:
              parsers[i].right_arc()
            else:
              parsers[i].shift()

        # Shift
        elif moves_argm[i] == 3:
          parsers[i].shift()

##Train and Test
Now that we have defined all our components, we are ready to train and test our model.

First we define our evaluation function. We use UAS (Unlabeled Accuracy Score) which is the percentage of correct arcs predicted over all the arcs.

In [ ]:
def evaluate(gold, preds):
  total = 0
  correct = 0

  for g, p in zip(gold, preds):
    for i in range(1,len(g)):
      total += 1
      if g[i] == p[i]:
        correct += 1

  return correct/total

We define the train loop.
The `train` function trains the given model on the dataset represented by the dataloader.
The `criterion` is used for computing the loss, the `optimizer` for updating the parameters of the model.
It calls `forward` function to compute scores for each possible action and for each parser configuration.

In [ ]:
def train(model, dataloader, criterion, optimizer):
  model.train()
  total_loss = 0
  count = 0

  for batch in dataloader:
    optimizer.zero_grad()
    sentences, paths, moves, trees = batch

    out = model(sentences, paths)
    labels = torch.tensor(sum(moves, [])).to(device) #sum(moves, []) flatten the array
    loss = criterion(out, labels)

    count +=1
    total_loss += loss.item()

    loss.backward()
    optimizer.step()

  return total_loss/count

We define test function in order to run inference.
In this case we evaluate the model on the dataloader.
- `infere` function is called in order to perform dependency parsing on the input sentences and obtain the prediction of the depedency trees.  
- `evaluate` for evaluation metrics on the predicted trees vs the gold trees



In [ ]:
def test(model, dataloader):
  model.eval()

  gold = []
  preds = []

  for batch in dataloader:
    sentences, paths, moves, trees = batch
    with torch.no_grad():
      pred = model.infere(sentences)

      gold += trees
      preds += pred

  return evaluate(gold, preds)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)
model = Net(device)
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LR)


for epoch in range(EPOCHS):
  avg_train_loss = train(model, train_dataloader, criterion, optimizer)
  val_uas = test(model, dev_dataloader)

  print("Epoch: {:3d} | avg_train_loss: {:5.3f} | dev_uas: {:5.3f} |".format( epoch, avg_train_loss, val_uas))

Device: cuda


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/rnn.py:71: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


Epoch:   0 | avg_train_loss: 1.041 | dev_uas: 0.257 |
Epoch:   1 | avg_train_loss: 0.918 | dev_uas: 0.278 |
Epoch:   2 | avg_train_loss: 0.889 | dev_uas: 0.282 |
Epoch:   3 | avg_train_loss: 0.872 | dev_uas: 0.296 |
Epoch:   4 | avg_train_loss: 0.859 | dev_uas: 0.295 |
Epoch:   5 | avg_train_loss: 0.848 | dev_uas: 0.292 |
Epoch:   6 | avg_train_loss: 0.840 | dev_uas: 0.297 |
Epoch:   7 | avg_train_loss: 0.834 | dev_uas: 0.297 |
Epoch:   8 | avg_train_loss: 0.828 | dev_uas: 0.298 |
Epoch:   9 | avg_train_loss: 0.822 | dev_uas: 0.305 |
Epoch:  10 | avg_train_loss: 0.818 | dev_uas: 0.300 |
Epoch:  11 | avg_train_loss: 0.814 | dev_uas: 0.302 |
Epoch:  12 | avg_train_loss: 0.812 | dev_uas: 0.306 |
Epoch:  13 | avg_train_loss: 0.809 | dev_uas: 0.303 |
Epoch:  14 | avg_train_loss: 0.805 | dev_uas: 0.306 |


In [ ]:
torch.save(model.state_dict(), "./bilstm.pt")


# 2nd Part
# BERT Model

In [ ]:
!pip install transformers
#!pip install datasets
!pip install evaluate
!pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 62.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 93.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 55.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 11.6 MB/s eta 0:00:00


In [ ]:
from transformers import AutoTokenizer, AutoModel

model_name = "bert-base-uncased"

model = AutoModel.from_pretrained("bert-base-uncased")
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
sequences = ["Using transformers is quite simple", "Natural Language Processing is the coolest area of AI", "BERT is an encoder-only model"]
batch = tokenizer(sequences, padding=True, return_tensors='pt')
print(batch["input_ids"], batch["attention_mask"], batch["token_type_ids"], sep="\n\n", end="\n\n")
print(tokenizer.batch_decode(batch["input_ids"]))

tensor([[  101,  2478, 19081,  2003,  3243,  3722,   102,     0,     0,     0,
             0,     0],
        [  101,  3019,  2653,  6364,  2003,  1996,  4658,  4355,  2181,  1997,
          9932,   102],
        [  101, 14324,  2003,  2019,  4372, 16044,  2099,  1011,  2069,  2944,
           102,     0]])

tensor([[1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0]])

tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

['[CLS] using transformers is quite simple [SEP] [PAD] [PAD] [PAD] [PAD] [PAD]', '[CLS] natural language processing is the coolest area of ai [SEP]', '[CLS] bert is an encoder - only model [SEP] [PAD]']


In [ ]:
model_output = model(**batch) # equivalent to model(input_ids=batch['input_ids'], attention_mask=batch['attention_mask'], token_type_ids=batch['token_type_ids'])
print(model_output.last_hidden_state)
print("batch_size x seq_len x hidden_dim", model_output.last_hidden_state.shape, sep="\n")

tensor([[[-2.0453e-01,  7.0865e-02,  1.5989e-01,  ..., -5.2576e-01,
          -8.6742e-02,  6.8585e-01],
         [ 2.4598e-01,  4.5985e-01, -1.4832e-01,  ..., -2.9959e-01,
          -6.3807e-02,  2.2089e-01],
         [ 1.6290e+00, -7.5314e-02, -1.6926e-01,  ..., -5.9800e-01,
          -2.4732e-01,  5.8838e-01],
         ...,
         [ 1.8744e-01, -1.4395e-01,  1.3122e-01,  ...,  5.4823e-02,
          -3.0255e-01,  4.1559e-01],
         [-4.2708e-01, -4.2683e-01,  1.7270e-01,  ...,  2.9482e-01,
           5.4690e-02,  4.9447e-01],
         [-9.1995e-02, -2.2785e-01,  9.8050e-02,  ...,  1.3341e-01,
          -1.4247e-01,  4.9720e-01]],

        [[-4.2831e-02,  2.8874e-02,  6.4191e-02,  ..., -1.5476e-01,
          -1.0478e-02,  5.8216e-01],
         [-1.0164e-01,  1.8912e-01, -5.2067e-01,  ..., -7.8897e-02,
           1.9872e-01,  2.9997e-01],
         [-5.4308e-01,  2.5865e-01,  5.8880e-01,  ..., -5.6930e-01,
          -3.8819e-01, -1.1054e-01],
         ...,
         [-3.7056e-01,  4

In [ ]:
model_output = model(**batch, output_hidden_states=True) # equivalent to model(input_ids=batch['input_ids'], attention_mask=batch['attention_mask'], token_type_ids=batch['token_type_ids'])
all_states = model_output.hidden_states # list of outputs from all transformer layers, layer 0, 1, 2, ...., 12 (layer 0 is the embedding layer)
print(len(all_states))
print(all_states[3]) # output of 4th layer

In [ ]:
# This is the function that we want to apply on the entire dataset
def tokenize(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)

In [ ]:
from transformers import AutoModelForSequenceClassification
BERT_model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased",num_labels=2)
print(BERT_model)

In [ ]:
def prepare_batch_transformer(batch_data, get_gold_path=False):
  data = [process_sample(s, get_gold_path=get_gold_path) for s in batch_data]

  sentences = [s[0] for s in data]
  paths = [s[1] for s in data]
  moves = [s[2] for s in data]
  trees = [s[3] for s in data]
  input_ids = [s[4] for s in data]
  connector =  [s[5] for s in data]
  attention_mask = [s[6] for s in data]
  return sentences, paths, moves, trees, input_ids, connector, attention_mask


  return sentences, paths, moves, trees, input_ids, connector, attention_mask

In [ ]:
from transformers import BertModel

class BERTNet(nn.Module):
    def __init__(self, device):
        super(BERTNet, self).__init__()
        self.device = device

        # initialize BERT
        self.bert = BertModel.from_pretrained("bert-base-uncased", output_hidden_states=True)

        # Freeze the first 3 layers
        for param in self.bert.encoder.layer[:3].parameters():
            param.requires_grad = False

        # initialize feedforward
        self.w1 = torch.nn.Linear(768, MLP_SIZE, bias=True) ## 768 is the dimension of the output of Bert Base uncased
        self.activation = torch.nn.Tanh()
        self.w2 = torch.nn.Linear(MLP_SIZE, 4, bias=True)
        self.softmax = torch.nn.Softmax(dim=-1)

        self.dropout = torch.nn.Dropout(DROPOUT)


  def forward(self, x, paths):
    # get the embeddings
    x = [self.dropout(self.embeddings(torch.tensor(i).to(self.device))) for i in x]

    # run the bi-lstm
    h = self.lstm_pass(x)

    # for each parser configuration that we need to score we arrange from the
    # output of the bi-lstm the correct input for the feedforward
    mlp_input = self.get_mlp_input(paths, h)

    # run the feedforward and get the scores for each possible action
    out = self.mlp(mlp_input)

    return out

  def lstm_pass(self, x):
    x = torch.nn.utils.rnn.pack_sequence(x, enforce_sorted=False)
    h, (h_0, c_0) = self.lstm(x)
    h, h_sizes = torch.nn.utils.rnn.pad_packed_sequence(h) # size h: (length_sentences, batch, output_hidden_units)
    return h

  def get_mlp_input(self, configurations, h):
    mlp_input = []
    zero_tensor = torch.zeros(2*LSTM_SIZE, requires_grad=False).to(self.device)
    for i in range(len(configurations)): # for every sentence in the batch
      for j in configurations[i]: # for each configuration of a sentence
        mlp_input.append(torch.cat([zero_tensor if j[0]==-1 else h[j[0]][i], zero_tensor if j[1]==-1 else h[j[1]][i], zero_tensor if j[2]==-1 else h[j[2]][i]]))
    mlp_input = torch.stack(mlp_input).to(self.device)
    return mlp_input

  def mlp(self, x):
    return self.softmax(self.w2(self.dropout(self.activation(self.w1(self.dropout(x))))))

  # we use this function at inference time. We run the parser and at each step
  # we pick as next move the one with the highest score assigned by the model
  def infere(self, x):

    parsers = [ArcEager(i) for i in x]

    x = [self.embeddings(torch.tensor(i).to(self.device)) for i in x]

    h = self.lstm_pass(x)

    while not self.parsed_all(parsers):
      # get the current configuration and score next moves
      configurations = self.get_configurations(parsers)
      mlp_input = self.get_mlp_input(configurations, h)
      mlp_out = self.mlp(mlp_input)
      # take the next parsing step
      self.parse_step(parsers, mlp_out)

    # return the predicted dependency tree
    return [parser.arcs for parser in parsers]

  def get_configurations(self, parsers):
    configurations = []

    for parser in parsers:
      if parser.is_tree_final():
        conf = [-1, -1, -1]
      else:
        conf = [parser.stack[len(parser.stack)-2], parser.stack[len(parser.stack)-1]]
        if len(parser.buffer) == 0:
          conf.append(-1)
        else:
          conf.append(parser.buffer[0])
      configurations.append([conf])

    return configurations

  def parsed_all(self, parsers):
    for parser in parsers:
      if not parser.is_tree_final():
        return False
    return True

  # In this function we select and perform the next move according to the scores obtained.
  # We need to be careful and select correct moves, e.g. don't do a shift if the buffer
  # is empty or a left arc if σ2 is the ROOT. For clarity sake we didn't implement
  # these checks in the parser so we must do them here. This renders the function quite ugly
  def parse_step(self, parsers, moves):
      moves_argm = moves.argmax(-1)
      for i in range(len(parsers)):
          if parsers[i].is_tree_final():
              continue
          else:
              # Left arc
              if moves_argm[i] == 0:
                  stack_len = len(parsers[i].stack)
                  if parsers[i].stack[-1] != 0 and len(parsers[i].buffer) > 0:
                      parsers[i].left_arc()
                  else:
                    if len(parsers[i].stack) >= 2 and len(parsers[i].buffer) > 0:
                      parsers[i].right_arc()
                    elif len(parsers[i].stack) >= 2:
                      parsers[i].reduce()
                    else:
                      parsers[i].shift()

              # Right arc
              elif moves_argm[i] == 1:
                  stack_len = len(parsers[i].stack)
                  if stack_len >= 2 and len(parsers[i].buffer) > 0:
                      parsers[i].right_arc()
                  else:
                      if parsers[i].stack[-1] != 0 and len(parsers[i].buffer) > 0:
                        parsers[i].left_arc()
                      elif len(parsers[i].stack) >= 2:
                        parsers[i].reduce()
                      else:
                        parsers[i].shift()

              # Shift
              elif moves_argm[i] == 2:
                  if len(parsers[i].buffer) > 0:
                      parsers[i].shift()
                  else:
                    if parsers[i].stack[-1] != 0 and len(parsers[i].buffer) > 0:
                        parsers[i].left_arc()
                    elif len(parsers[i].stack) >= 2 and len(parsers[i].buffer) > 0:
                      parsers[i].right_arc()
                    elif len(parsers[i].stack) >= 2:
                      parsers[i].reduce()

              # Reduce
              elif moves_argm[i] == 3:
                  if len(parsers[i].stack) >= 2:
                      parsers[i].reduce()
                  else:
                    if parsers[i].stack[-1] != 0 and len(parsers[i].buffer) > 0:
                        parsers[i].left_arc()
                    elif len(parsers[i].stack) >= 2 and len(parsers[i].buffer) > 0:
                      parsers[i].right_arc()
                    else:
                      parsers[i].shift()